# 📘 DevOps 업무전환 - Story 포인트 측정
- **목적**: DevOps 전환을 위해 생성된 스토리에서 스토리포인트 산정
- **입력 데이터**: generate_story_of_devops.ipynb의 결과물
- **주요 기능**
  - 스토리 포인트 참고 자료 로드
  - 스토리별 스토리 포인트 산정 
- **작성자**: 김동영
- **작성일**: 2025-08-05

# 0. 환경 설정

In [ ]:
import os, sys
sys.path.append(os.path.abspath("../src"))

import dotenv
import json

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from story_point.services import StoryPointEstimationAgent
# from epic.prompts import TASK_TO_EPIC_CONVERTER_PROMPT

In [ ]:
# .env 환경번수 load
dotenv.load_dotenv('../.env')

# 1. Story Point reference load and Agent 인스턴스 생성

In [ ]:
# 스토리 생성 llm 모델 생성
story_point_agent = StoryPointEstimationAgent(
    openai_api_key = os.getenv("OPENAI_API_KEY"),csv_file_path='../data/reference_stories_250807.csv'
)

In [ ]:
STORY_POINT_ESTIMATION_PROMPT = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template(
        """
        [Role] 너는 숙련된 스크럼 마스터다.
        [Goal] 2주 단위 스프린트를 기반으로 한 애자일 방법론에서, 주어진 에픽정보와 Story를 기반으로 각 Story의 업무 리소스 산정 단위인 StoryPoint를 도출하고 업무 계획을 수립하는 데 도움을 주는 것.
        [Instruction] 주어진 Epic 정보와 Story 내용을 기반으로, 각 Story의 StoryPoint를 추정하고, 그 이유를 상세하게 설명해.
        [Constraints]
          - 출력은 반드시 [Output Format]을 따라야 한다.
          - StoryPoint는 반드시 1, 2, 3, 5, 8 중 하나의 값을 가진다.
          - 불필요한 설명, 주석, 마크다운 없이 JSON만 출력한다.
          - Story가 1개이더라도 반드시 [Output Format]과 동일한 JSON 배열 형식으로 출력한다.
          - 스토리 포인트 추정 기준
            - **1 포인트**: 매우 간단한 작업 (1-2시간)
              - 단순 설정 변경, 텍스트 수정, 기존 기능 복사
            - **2 포인트**: 간단한 작업 (반나절)
              - 간단한 CRUD 작업, 기존 컴포넌트 재사용
            - **3 포인트**: 보통 작업 (1일, 6-8시간)
              - 새로운 화면 개발, 중간 복잡도의 비즈니스 로직
            - **5 포인트**: 복잡한 작업 (2-3일, 12-24시간)
              - 복잡한 비즈니스 로직, 외부 시스템 연동, 여러 컴포넌트 간 협업
            - **8 포인트**: 매우 복잡한 작업 (1주일, 32-40시간)
              - 아키텍처 변경, 대규모 리팩토링, 신규 기술 도입, 높은 불확실성
        [Process]
          1 → 2 → 3 → 4의 순서로 Story의 복잡도를 분석하고 추정 방법을 선택한 뒤, 최종 StoryPoint를 결정한다.
          1. 참고 스토리 분석
            - 같은 영역의 참고 스토리가 있는 경우: **직접 비교**
            - 같은 영역의 참고 스토리가 없는 경우: **다른 영역 스토리 활용**
            - 참고할 스토리가 없는 경우: **해당 스토리를 분석하여 임의로 산정**

          2. 복잡도 분석
            다음 요소들을 종합적으로 평가
            2.1 기술적 복잡도
              - 구현 난이도 (새로운 기술/라이브러리 사용 여부)
              - 코드 변경 범위 (신규 개발 vs 기존 수정)
              - 테스트 복잡도 (단위/통합/E2E 테스트 범위)
              - 성능 고려사항 (최적화 필요 여부)
            2.2 비즈니스 복잡도
              - 요구사항 명확성 및 완성도
              - 비즈니스 로직 복잡도
              - 사용자 인터페이스 복잡도
              - 데이터 처리 복잡도
              - 인수 조건(acceptance_criteria) 복잡도 및 개수
            2.3 위험 요소
              - 불확실성 (요구사항 변경 가능성)
              - 의존성 (다른 팀/시스템과의 의존도)
              - 학습 곡선 (새로운 기술/도메인 학습 필요)
              - 블로커 가능성 (외부 요인으로 인한 지연 위험)

          3. 추정 방법 선택
            Case1 직접 비교 (같은 영역 스토리가 있는 경우)
              1. 같은 영역의 참고 스토리들과 복잡도 비교
              2. 가장 유사한 스토리 식별
              3. 상대적 차이를 고려하여 포인트 조정
            Case2 다른 영역 스토리 활용 (같은 영역 스토리가 없는 경우)
              1. 모든 영역의 스토리 중 유사한 복잡도 패턴 찾기
              2. 영역 차이를 고려한 복잡도 보정
              3. 기술적/비즈니스적 유사성이 높은 스토리 우선 참조

          4. 최종 포인트 결정
            - 반드시 피보나치 수열 값(1, 2, 3, 5, 8) 중 선택
            - 두 포인트 사이에서 애매한 경우 더 높은 포인트 선택 (안전 마진)

        [Output Format]
         [
          {{
            "story_title": "스토리 제목",
            "estimated_point": 3,
            "domain": "frontend|backend|devops|data",
            "estimation_method": "same_area|cross_area",
            "reasoning": "상세한 추정 근거 (왜 이 포인트인지 논리적 설명)",
            "complexity_factors": [
              "기술적 복잡도 요소",
              "비즈니스 복잡도 요소",
              "위험 요소"
            ],
            "similar_stories": [
              "유사한 참고 스토리 제목들 (영역 구분 없이)"
            ],
            "confidence_level": "high|medium|low",
            "assumptions": [
              "추정 시 가정한 사항들"
            ],
            "risks": [
              "예상되는 위험 요소들"
            ]
          }}
        ]
        """
    ),
    HumanMessagePromptTemplate.from_template(
        """
        [Input] {user_input}
        [Context]
          epic info : {epic_info}
          story info : {story_info}
          reference_stories : {reference_stories}
        """
    )
    ]
)


In [ ]:
story_point_agent.csv_file_path

In [ ]:
ci_cd_story_list = [
  {
    "title": "CI/CD 자동화 도구 선택",
    "description": "GitHub Actions 또는 Jenkins 중 하나를 선택하여 CI/CD 자동화 도구를 결정한다.",
    "acceptance_criteria": [
      "CI/CD 도구 선택에 대한 문서화가 완료된다.",
      "선택된 도구의 장단점 분석이 포함된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "GitHub Actions", "Jenkins"]
  },
  {
    "title": "CI/CD 파이프라인 기본 구성",
    "description": "선택된 CI/CD 도구를 사용하여 기본 파이프라인을 구성한다.(Bnviit IOL 기준)",
    "acceptance_criteria": [
      "기본 파이프라인이 성공적으로 실행된다.",
      "파이프라인 구성에 대한 문서가 작성된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "파이프라인", "자동화"]
  },
  {
  "title": "PR 시 자동 코드 검증 및 품질 점검 설정",
  "description": "GitHub Actions 등 CI 도구를 통해 PR 생성 시 Lint, Formatting, Unit Test, Build, Code Quality Check가 자동으로 실행되도록 설정한다.(Bnviit IOL 기준)",
  "acceptance_criteria": [
    "PR 생성 시 Lint가 자동으로 실행된다.",
    "PR 생성 시 코드 Formatting이 자동으로 검증된다.",
    "PR 생성 시 Unit Test가 자동으로 실행된다.",
    "PR 생성 시 Build가 자동으로 실행된다.",
    "PR 생성 시 Code Quality Check가 자동으로 실행된다."
  ],
  "domain": "devops",
  "story_type": "feature",
  "tags": ["CI/CD", "Lint", "Unit Test", "Build", "Code Quality", "GitHub Actions"]
},
  {
    "title": "Dockerfile 오류 및 종속성 검증 설정",
    "description": "Dockerfile 실행 오류 및 종속성 검증을 CI/CD 파이프라인에 포함시킨다.(Bnviit IOL 기준)",
    "acceptance_criteria": [
      "Dockerfile 실행 오류가 감지될 경우 알림이 발생한다.",
      "종속성 검증이 성공적으로 수행된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["Docker", "CI/CD", "검증"]
  },
  {
    "title": "CI/CD 결과 알림 설정",
    "description": "CI/CD 결과를 Slack, 이메일, 웹훅으로 알림받도록 설정한다.(Bnviit IOL 기준)",
    "acceptance_criteria": [
      "CI/CD 결과가 Slack으로 성공적으로 전송된다.",
      "CI/CD 결과가 이메일로 성공적으로 전송된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["알림", "Slack", "이메일"]
  },
  {
    "title": "비앤빛 비젼(Java) 프로젝트에 CI/CD 적용",
    "description": "비앤빛 비젼(Java 기반)의 백엔드 프로젝트에 CI/CD 자동화 구성을 적용한다. Java 프로젝트 특성에 맞게 Maven/Gradle 기반 빌드 및 테스트 설정을 포함한다.",
    "acceptance_criteria": [
      "Java 기반 빌드 및 테스트 파이프라인이 구성된다.",
      "PR 생성 시 자동으로 Lint, Test, Build가 실행된다.",
      "Slack 또는 이메일을 통한 알림이 설정된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "Java", "비앤빛비젼", "GitHub Actions", "Maven", "Gradle"]
  },
  {
    "title": "브리즘 마이오피아 프로젝트에 CI/CD 적용",
    "description": "브리즘 마이오피아 프로젝트에 맞는 CI/CD 파이프라인을 구성하고, Python 기반 프로젝트 특성에 맞는 Lint, Test, Build 설정을 포함한다.",
    "acceptance_criteria": [
      "Python 기반 빌드 및 테스트 파이프라인이 구성된다.",
      "PR 생성 시 자동으로 Lint, Test, Build가 실행된다.",
      "CI 결과가 Slack 또는 이메일로 전송된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "Python", "브리즘", "GitHub Actions"]
  },
  {
    "title": "비앤빛 OCR 프로젝트에 CI/CD 적용",
    "description": "비앤빛 OCR 서비스에 CI/CD 파이프라인을 구축하고, 모델 추론 로직의 검증과 이미지 처리 테스트를 자동화한다.",
    "acceptance_criteria": [
      "OCR 모델 추론에 대한 테스트가 CI에 포함된다.",
      "PR 생성 시 Lint, Unit Test, Build가 자동 실행된다.",
      "CI/CD 결과에 대한 알림이 설정된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "OCR", "이미지처리", "비앤빛OCR", "GitHub Actions"]
  },
  {
    "title": "비앤빛 App 프로젝트에 CI/CD 적용",
    "description": "비앤빛 App (모바일 또는 웹 프론트) 프로젝트에 CI/CD 자동화 설정을 적용하고, 프론트엔드 빌드 및 테스트 수행을 포함한다.",
    "acceptance_criteria": [
      "프론트엔드 빌드 및 테스트가 자동으로 실행된다.",
      "PR 생성 시 Lint, Test, Build가 포함된다.",
      "Slack 알림 또는 GitHub status check가 설정된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "프론트엔드", "비앤빛App", "React", "GitHub Actions"]
  }
]

In [ ]:
ci_cd_epic_info ={
    "title": "CI/CD 자동화 구성",
    "description": "GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.",
    "business_value": "개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.",
    "priority": "High",
    "acceptance_criteria": [
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.",
      "Dockerfile 실행 오류 및 종속성 검증이 포함된다.",
      "CI/CD 결과가 Slack/이메일/웹훅으로 알림된다."
    ],
    "included_tasks": [
      "GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성",
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행",
      "Dockerfile 실행 오류 및 종속성 검증 포함",
      "CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동"
    ]
  },

In [ ]:
infra_epics_info

In [ ]:
story_point_agent.get_reference_stories_by_domain('DevOps')

In [ ]:
story_point_by_story = {}

for story in ci_cd_story_list[:]:
    story_point_by_story[story['title']] = {}

    estimated_story_point_str = story_point_agent._generate_estimations_with_llm(
        prompt = STORY_POINT_ESTIMATION_PROMPT,
        user_input = '',
        story_info= story,
        epic_info=ci_cd_epic_info,
        reference_stories=story_point_agent.get_reference_stories_by_domain('DevOps')
    )
    parsed = json.loads(estimated_story_point_str)
    story_point_by_story[story['title']] = parsed


In [ ]:
story_point_by_story

In [ ]:
story_point_agent.get_reference_stories_by_domain('DevOps')

In [ ]:
infra_resource_list = [

   {
      "title":"미사용 AWS 리소스 식별 및 정리",
      "description":"기존 사용 중인 AWS 리소스 중 미사용 리소스를 식별하고 정리하는 작업",
      "acceptance_criteria":[
         "모든 미사용 AWS 리소스가 식별되었음",
         "식별된 미사용 리소스가 정리되었음",
         "정리된 리소스 목록이 문서화되어 있음"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "AWS",
         "리소스 관리",
         "비용 최적화"
      ]
   },
   {
      "title":"비용 최적화 방안 문서화",
      "description":"비용 최적화를 위한 다양한 방안을 분석하고 문서화하는 작업",
      "acceptance_criteria":[
         "비용 최적화 방안이 3가지 이상 제안되었음",
         "각 방안의 장단점이 명시되었음",
         "문서화된 내용이 팀과 공유되었음"
      ],
      "domain":"backend",
      "story_type":"documentation",
      "tags":[
         "비용 최적화",
         "AWS",
         "문서화"
      ]
   },
   {
      "title":"리소스 태깅 정책 적용",
      "description":"AWS 리소스에 대한 태깅 정책을 수립하고 적용하는 작업",
      "acceptance_criteria":[
         "리소스 태깅 정책이 정의되었음",
         "정의된 태깅 정책이 모든 리소스에 적용되었음",
         "예산 분류가 가능하도록 태깅이 완료되었음"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "AWS",
         "리소스 태깅",
         "비용 관리"
      ]
   },
   {
      "title":"Auto Scaling 및 예약 인스턴스 설정",
      "description":"비용 최적화를 위해 Auto Scaling 및 예약 인스턴스를 설정하는 작업",
      "acceptance_criteria":[
         "Auto Scaling이 설정되어 트래픽 변화에 대응할 수 있음",
         "예약 인스턴스가 적절히 선택되어 비용 절감 효과가 있음",
         "설정 내용이 문서화되어 팀과 공유되었음"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "AWS",
         "Auto Scaling",
         "비용 최적화"
      ]
   },
   {
      "title":"Spot 인스턴스 활용 방안 연구",
      "description":"비용 절감을 위해 Spot 인스턴스를 활용하는 방안을 연구하는 작업",
      "acceptance_criteria":[
         "Spot 인스턴스의 활용 가능성이 분석되었음",
         "적용 가능한 서비스 목록이 작성되었음",
         "연구 결과가 문서화되어 팀과 공유되었음"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "AWS",
         "Spot 인스턴스",
         "비용 최적화"
      ]
   },
   {
      "title":"인스턴스 사양 관리 방안 수립",
      "description":"비용 최적화를 위한 인스턴스 사양 관리 방안을 수립하는 작업",
      "acceptance_criteria":[
         "현재 사용 중인 인스턴스 사양이 분석되었음",
         "최적의 인스턴스 사양이 제안되었음",
         "사양 관리 방안이 문서화되어 팀과 공유되었음"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "AWS",
         "인스턴스 관리",
         "비용 최적화"
      ]
   }
]

In [ ]:
infra_resource_epic_info = {
      "title":"AWS 리소스 관리 및 비용 최적화",
      "description":"기존 사용 중인 AWS 리소스를 관리하고 비용 최적화 방안을 모색하는 작업",
      "business_value":"비용 절감 및 리소스 효율성 향상",
      "priority":"High",
      "acceptance_criteria":[
         "모든 미사용 AWS 리소스가 식별되고 정리되었음",
         "비용 최적화 방안이 문서화되어 있음",
         "리소스 태깅 정책이 적용되어 예산 분류가 가능함"
      ],
      "included_tasks":[
         "EC2, ECS, EKS, S3, RDS, ALB 등 기존 사용 중인 AWS 리소스를 관리하고 주기적으로 미사용 리소스를 정리하고 분석하여 비용 최적화 방안을 모색",
         "환경별 구성 분리, 리소스 태깅",
         "Auto Scaling, 예약 인스턴스, spot 인스턴스, 인스턴스 사양 관리 등 비용 최적화 방안을 찾음",
         "리소스 태깅 정책 적용 및 예산 분류 가능하게 구성"
      ]
   }

In [ ]:
certificate_story_list = [
   {
      "title":"SSL 인증서 자동 갱신 설정",
      "description":"ACM 또는 certbot을 사용하여 SSL 인증서가 자동으로 갱신되도록 설정한다.",
      "acceptance_criteria":[
         "ACM을 통해 SSL 인증서가 자동으로 갱신되도록 설정됨",
         "certbot을 사용하여 SSL 인증서가 자동으로 갱신되도록 설정됨",
         "자동 갱신 테스트를 통해 인증서가 만료되지 않음을 확인함"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "SSL",
         "ACM",
         "certbot",
         "자동 갱신"
      ]
   },
   {
      "title":"도메인 관리 프로세스 문서화",
      "description":"도메인 관리 프로세스를 문서화하여 팀원들이 쉽게 이해하고 따를 수 있도록 한다.",
      "acceptance_criteria":[
         "도메인 등록, 갱신, 이전 절차가 문서화됨",
         "문서가 팀원들에게 공유되고 피드백을 받음",
         "문서가 최신 상태로 유지되도록 관리 계획 수립"
      ],
      "domain":"documentation",
      "story_type":"documentation",
      "tags":[
         "도메인 관리",
         "문서화",
         "프로세스"
      ]
   },
   {
      "title":"Route53을 통한 도메인 관리 설정",
      "description":"AWS Route53을 사용하여 도메인 관리 프로세스를 설정하고 운영한다.",
      "acceptance_criteria":[
         "Route53에서 도메인이 성공적으로 등록됨",
         "DNS 레코드가 올바르게 설정됨",
         "도메인 관리 대시보드가 팀에 공유됨"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "Route53",
         "AWS",
         "도메인 관리"
      ]
   }
]

In [ ]:
certificate_epic_info =    {
      "title":"인증서 관리 및 자동 갱신",
      "description":"SSL 인증서 및 도메인 관리와 관련된 업무",
      "business_value":"서비스의 보안성 강화 및 운영 효율성 증대",
      "priority":"Medium",
      "acceptance_criteria":[
         "모든 SSL 인증서가 자동으로 갱신되도록 설정됨",
         "도메인 관리 프로세스가 문서화되어 있음"
      ],
      "included_tasks":[
         "인증서 자동 갱신 (ACM, certbot 등)",
         "Route53, 도메인, SSL 인증서 갱신 관리"
      ]
   }

In [ ]:
iam_story_list = [
   {
      "title":"IAM Role 정리 및 관리",
      "description":"기존 IAM Role을 분석하고 불필요한 Role을 제거하며, 필요한 Role을 재구성하여 최소 권한 접근 제어를 구현한다.",
      "acceptance_criteria":[
         "모든 기존 IAM Role이 분석되어 정리됨",
         "불필요한 IAM Role이 삭제됨",
         "최소 권한 접근 제어를 위한 새로운 IAM Role이 생성됨"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "IAM",
         "보안",
         "접근 제어"
      ]
   },
   {
      "title":"정책 문서화",
      "description":"접근 제어 정책을 문서화하여 모든 팀원이 이해할 수 있도록 한다.",
      "acceptance_criteria":[
         "접근 제어 정책이 문서화되어 팀원에게 공유됨",
         "문서화된 정책이 검토 및 승인됨",
         "정책 문서가 중앙 저장소에 저장됨"
      ],
      "domain":"documentation",
      "story_type":"documentation",
      "tags":[
         "정책",
         "문서화",
         "접근 제어"
      ]
   },
   {
      "title":"파트너사 접근 제어 정책 관리",
      "description":"파트너사와의 역할 분리 및 접근 제어 정책을 설정하고 관리한다.",
      "acceptance_criteria":[
         "파트너사와의 역할이 명확히 정의됨",
         "접속 권한 및 키 회수 정책이 수립됨",
         "파트너사에 대한 접근 제어가 문서화됨"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "파트너사",
         "접근 제어",
         "정책 관리"
      ]
   }
]

In [ ]:
iam_epic_info =    {
      "title":"IAM 및 접근 제어 관리",
      "description":"IAM Role 기반의 최소 권한 접근 제어 및 정책 관리",
      "business_value":"보안 강화 및 접근 관리 효율성 증대",
      "priority":"High",
      "acceptance_criteria":[
         "모든 IAM role, 정책, 사용자가 정리되고 관리됨",
         "접근 제어 정책이 문서화되어 있음"
      ],
      "included_tasks":[
         "IAM Role 기반 최소 권한 접근 제어를 하도록 구성하고 기존 IAM role, 정책, 사용자를 정리하고 관리",
         "파트너사와의 역할 분리 및 접근 제어 정책 관리 (접속 권한, 키 회수 등)"
      ]
   }

In [ ]:
security_rule_story_list = [
   {
      "title":"인프라 리소스 네이밍 룰 정의",
      "description":"인프라 리소스의 네이밍 규칙을 정의하고 문서화한다.",
      "acceptance_criteria":[
         "네이밍 룰이 문서화되어 팀원에게 공유됨",
         "정의된 네이밍 룰이 모든 인프라 리소스에 적용됨",
         "주기적으로 네이밍 룰이 검토되고 업데이트됨"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "보안 정책",
         "인프라",
         "네이밍 규칙"
      ]
   },
   {
      "title":"키 관리 방법 정의",
      "description":"인프라 리소스에서 사용할 키 관리 방법을 정의하고 문서화한다.",
      "acceptance_criteria":[
         "키 관리 방법이 문서화되어 팀원에게 공유됨",
         "정의된 키 관리 방법이 모든 관련 시스템에 적용됨",
         "주기적으로 키 관리 방법이 검토되고 업데이트됨"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "보안 정책",
         "키 관리",
         "인프라"
      ]
   },
   {
      "title":"MFA 정책 정의",
      "description":"다단계 인증(MFA) 정책을 정의하고 문서화한다.",
      "acceptance_criteria":[
         "MFA 정책이 문서화되어 팀원에게 공유됨",
         "정의된 MFA 정책이 모든 사용자 계정에 적용됨",
         "주기적으로 MFA 정책이 검토되고 업데이트됨"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "보안 정책",
         "MFA",
         "인프라"
      ]
   },
   {
      "title":"보안 그룹 정책 정의",
      "description":"인프라 리소스의 보안 그룹 정책을 정의하고 문서화한다.",
      "acceptance_criteria":[
         "보안 그룹 정책이 문서화되어 팀원에게 공유됨",
         "정의된 보안 그룹 정책이 모든 관련 리소스에 적용됨",
         "주기적으로 보안 그룹 정책이 검토되고 업데이트됨"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "보안 정책",
         "보안 그룹",
         "인프라"
      ]
   },
   {
      "title":"비용 리포트 기준 정의",
      "description":"주기적 비용 리포트 및 미사용 자원 정리 기준을 정의하고 문서화한다.",
      "acceptance_criteria":[
         "비용 리포트 기준이 문서화되어 팀원에게 공유됨",
         "정의된 기준에 따라 비용 리포트가 생성됨",
         "주기적으로 비용 리포트 기준이 검토되고 업데이트됨"
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "비용 관리",
         "보안 정책",
         "인프라"
      ]
   }
]

In [ ]:
security_rule_epic_info = {
      "title":"보안 정책 및 기준 설정",
      "description":"인프라 리소스와 관련된 보안 정책 및 기준을 정의하는 작업",
      "business_value":"보안 리스크 감소 및 규정 준수",
      "priority":"Medium",
      "acceptance_criteria":[
         "모든 보안 정책이 문서화되어 있으며 주기적으로 검토됨"
      ],
      "included_tasks":[
         "정책 생성 (인프라 리소스 네이밍 룰 정의, 키 관리방법, MFA, 보안 그룹 등 보안 정책 명시, 주기적 비용 리포트 및 미사용 자원 정리 기준, 민감정보 관리 기준 정책 등)"
      ]
   }

In [ ]:
metric_story_list = [
   {
      "title":"메트릭 대시보드 구성",
      "description":"Prometheus와 Grafana를 사용하여 서비스의 메트릭을 시각화하는 대시보드를 구성한다.",
      "acceptance_criteria":[
         "Prometheus와 Grafana를 통해 메트릭 대시보드가 성공적으로 구성됨.",
         "대시보드에서 CPU 사용량, 메모리 사용량, 요청 수 등의 주요 메트릭이 실시간으로 표시됨.",
         "대시보드의 시각화가 사용자 친화적이며, 필요한 필터링 기능이 포함됨."
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "Prometheus",
         "Grafana",
         "모니터링"
      ]
   },
   {
      "title":"서비스 로그 수집 및 시각화",
      "description":"Loki와 Grafana를 통해 서비스 로그를 수집하고 시각화하는 작업을 수행한다.",
      "acceptance_criteria":[
         "Loki와 Grafana를 통해 서비스 로그가 성공적으로 수집됨.",
         "대시보드에서 로그가 실시간으로 시각화되고, 검색 및 필터링 기능이 제공됨.",
         "로그 데이터의 저장 및 조회 성능이 최적화됨."
      ],
      "domain":"backend",
      "story_type":"feature",
      "tags":[
         "Loki",
         "Grafana",
         "로그 수집"
      ]
   },
   {
      "title":"대시보드 주요 메트릭 설정",
      "description":"대시보드에서 표시할 주요 메트릭을 정의하고 설정한다.",
      "acceptance_criteria":[
         "대시보드에 표시할 주요 메트릭 목록이 정의됨.",
         "각 메트릭의 시각화 방식이 결정되고 적용됨.",
         "사용자가 메트릭을 쉽게 이해할 수 있도록 설명이 추가됨."
      ],
      "domain":"frontend",
      "story_type":"feature",
      "tags":[
         "대시보드",
         "메트릭",
         "UI"
      ]
   },
	 {
    "title": "모니터링 및 로그 대시보드 적용 또는 수정 - 비앤빛 비젼(Java)",
    "description": "비앤빛 비젼(Java) 서비스에 Prometheus, Loki, Grafana를 이용한 메트릭 및 로그 대시보드를 적용한다.",
    "acceptance_criteria": [
      "Prometheus 메트릭 수집이 비앤빛 비젼(Java) 서비스에 적용됨",
      "Loki 로그 수집이 비앤빛 비젼(Java) 서비스에 적용됨",
      "Grafana 대시보드에서 해당 서비스 메트릭과 로그가 정상 표시됨"
    ],
    "domain": "backend",
    "story_type": "feature",
    "tags": ["Prometheus", "Grafana", "Loki", "모니터링", "로그 수집", "대시보드", "비앤빛 비젼"]
  },
  {
    "title": "모니터링 및 로그 대시보드 적용 또는 수정 - 브리즘 마이오피아",
    "description": "브리즘 마이오피아 서비스에 Prometheus, Loki, Grafana를 이용한 메트릭 및 로그 대시보드를 적용한다.",
    "acceptance_criteria": [
      "Prometheus 메트릭 수집이 브리즘 마이오피아 서비스에 적용됨",
      "Loki 로그 수집이 브리즘 마이오피아 서비스에 적용됨",
      "Grafana 대시보드에서 해당 서비스 메트릭과 로그가 정상 표시됨"
    ],
    "domain": "backend",
    "story_type": "feature",
    "tags": ["Prometheus", "Grafana", "Loki", "모니터링", "로그 수집", "대시보드", "브리즘 마이오피아"]
  },
  {
    "title": "모니터링 및 로그 대시보드 적용 또는 수정 - 비앤빛 IOL",
    "description": "비앤빛 IOL 서비스에 Prometheus, Loki, Grafana를 이용한 메트릭 및 로그 대시보드를 적용한다.",
    "acceptance_criteria": [
      "Prometheus 메트릭 수집이 비앤빛 IOL 서비스에 적용됨",
      "Loki 로그 수집이 비앤빛 IOL 서비스에 적용됨",
      "Grafana 대시보드에서 해당 서비스 메트릭과 로그가 정상 표시됨"
    ],
    "domain": "backend",
    "story_type": "feature",
    "tags": ["Prometheus", "Grafana", "Loki", "모니터링", "로그 수집", "대시보드", "비앤빛 IOL"]
  },
  {
    "title": "모니터링 및 로그 대시보드 적용 또는 수정 - 비앤빛 OCR",
    "description": "비앤빛 OCR 서비스에 Prometheus, Loki, Grafana를 이용한 메트릭 및 로그 대시보드를 적용한다.",
    "acceptance_criteria": [
      "Prometheus 메트릭 수집이 비앤빛 OCR 서비스에 적용됨",
      "Loki 로그 수집이 비앤빛 OCR 서비스에 적용됨",
      "Grafana 대시보드에서 해당 서비스 메트릭과 로그가 정상 표시됨"
    ],
    "domain": "backend",
    "story_type": "feature",
    "tags": ["Prometheus", "Grafana", "Loki", "모니터링", "로그 수집", "대시보드", "비앤빛 OCR"]
  },
  {
    "title": "모니터링 및 로그 대시보드 적용 또는 수정 - 비앤빛 App",
    "description": "비앤빛 App 서비스에 Prometheus, Loki, Grafana를 이용한 메트릭 및 로그 대시보드를 적용한다.",
    "acceptance_criteria": [
      "Prometheus 메트릭 수집이 비앤빛 App 서비스에 적용됨",
      "Loki 로그 수집이 비앤빛 App 서비스에 적용됨",
      "Grafana 대시보드에서 해당 서비스 메트릭과 로그가 정상 표시됨"
    ],
    "domain": "backend",
    "story_type": "feature",
    "tags": ["Prometheus", "Grafana", "Loki", "모니터링", "로그 수집", "대시보드", "비앤빛 App"]
  }
]

In [ ]:
metric_epic_info = {
      "title":"메트릭 및 로그 대시보드 구성 ",
      "description":"Prometheus와 Grafana를 활용하여 메트릭 대시보드를 구성하고, Loki와 Grafana를 통해 서비스 로그를 수집 및 시각화하는 작업.",
      "business_value":"서비스의 성능 및 상태를 실시간으로 모니터링하여 운영 효율성을 향상.",
      "priority":"High",
      "acceptance_criteria":[
         "Prometheus와 Grafana를 통해 메트릭 대시보드가 성공적으로 구성됨.",
         "Loki와 Grafana를 통해 서비스 로그가 수집되고 시각화됨.",
         "대시보드에서 주요 메트릭 및 로그가 실시간으로 표시됨."
      ],
      "included_tasks":[
         "Prometheus + Grafana로 메트릭 대시보드 구성",
         "Loki + Grafana로 서비스 중인 서버의 로그 수집 및 시각화"
      ]
   }

In [ ]:
manage_log_story_list = [{'title': '로그 스키마 및 레벨 통일',
   'description': '서비스 로그의 스키마와 레벨을 JSON 기반으로 통일하는 작업.',
   'acceptance_criteria': ['모든 서비스의 로그 스키마가 JSON 형식으로 통일됨.',
    '로그 레벨이 DEBUG, INFO, WARN, ERROR로 표준화됨.',
    '통일된 로그 스키마 및 레벨에 대한 문서가 작성됨.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['로그 관리', 'JSON', '백엔드']},
  {'title': '로그 보존 주기 및 저장 정책 수립',
   'description': '서비스 로그의 보존 주기 및 저장 정책을 정의하는 작업.',
   'acceptance_criteria': ['각 서비스별 로그 보존 주기가 정의됨.',
    '로그 저장 정책이 문서화되어 팀과 공유됨.',
    '정의된 정책이 시스템에 적용되어 로그가 자동으로 관리됨.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['로그 관리', '정책 수립', '백엔드']},
  {'title': '서비스별 로그 레벨 및 포맷 표준화 문서 작성',
   'description': '서비스별 로그 레벨 및 포맷에 대한 표준화 문서를 작성하는 작업.',
   'acceptance_criteria': ['모든 서비스의 로그 레벨 및 포맷이 문서화됨.',
    '문서가 팀의 모든 구성원에게 공유됨.',
    '문서에 대한 피드백이 수집되어 최종본이 작성됨.'],
   'domain': 'documentation',
   'story_type': 'documentation',
   'tags': ['로그 관리', '문서화', '기준 설정']}]

In [ ]:
manage_log_epic_info =    {
      "title":"로그 관리 및 정책 수립",
      "description":"서비스 로그의 수집, 저장, 보존 정책을 수립하고, 로그 스키마 및 레벨을 통일하는 작업.",
      "business_value":"일관된 로그 관리로 데이터 분석 및 문제 해결의 효율성을 높임.",
      "priority":"High",
      "acceptance_criteria":[
         "로그 스키마와 레벨이 JSON 기반으로 통일됨.",
         "로그 보존 주기 및 저장 정책이 정의됨.",
         "서비스별 로그 레벨 및 포맷 표준화 문서가 작성됨."
      ],
      "included_tasks":[
         "로그 스키마/레벨 통일, JSON 기반 구조로 통일",
         "로그 보존 주기 및 저장 정책 수립",
         "정책 생성(서비스별 로그 레벨 및 포맷 표준화, 알람 임계치 설정 기준 문서화, 로그 보관 주기 및 백업 연계 정의 등)"
      ]
   }

In [ ]:
alarm_story_list = [{'title': 'Slack 알림 연동 구현',
   'description': 'Slack으로 알림을 전송하는 기능을 구현한다.',
   'acceptance_criteria': ['Slack으로의 알림 연동이 성공적으로 구현됨.',
    '알림 메시지가 지정된 채널에 정확히 전송됨.',
    '알림 전송 실패 시 에러 로그가 기록됨.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['Slack', '알림', '백엔드']},
  {'title': '로그 관리 도구 연동 방안 문서화',
   'description': 'Datadog, ELK, CloudWatch Logs 등과의 연동 방안을 문서화한다.',
   'acceptance_criteria': ['Datadog, ELK, CloudWatch Logs와의 연동 방안이 문서화됨.',
    '문서에는 각 도구의 설정 방법과 사용 예시가 포함됨.',
    '문서가 팀원들에게 공유되고 피드백을 받음.'],
   'domain': 'devops',
   'story_type': 'documentation',
   'tags': ['로그 관리', '문서화', 'DevOps']},
  {'title': '주요 이벤트 자동 수집 기능 구현',
   'description': '에러, 로그인, 배포 등의 주요 이벤트를 자동으로 수집하는 기능을 구현한다.',
   'acceptance_criteria': ['주요 이벤트(에러, 로그인, 배포 등)가 자동으로 수집되고 알림이 전송됨.',
    '수집된 이벤트가 로그 시스템에 정확히 기록됨.',
    '이벤트 수집 및 알림 전송의 성공 여부가 모니터링 가능함.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['이벤트 수집', '알림', '백엔드']}]

In [ ]:
alarm_epic_info =   {
      "title":"알림 및 통합 시스템 구축",
      "description":"Slack과의 알림 연동 및 다양한 로그 관리 도구와의 연동 방안을 모색하는 작업.",
      "business_value":"신속한 문제 대응 및 팀 간의 원활한 커뮤니케이션을 지원.",
      "priority":"Medium",
      "acceptance_criteria":[
         "Slack으로의 알림 연동이 성공적으로 구현됨.",
         "Datadog, ELK, CloudWatch Logs 등과의 연동 방안이 문서화됨.",
         "주요 이벤트(에러, 로그인, 배포 등)가 자동으로 수집되고 알림이 전송됨."
      ],
      "included_tasks":[
         "Slack으로 알림연동",
         "Datadog, ELK, CloudWatch Logs 등과 연동하는 방안 모색",
         "주요 이벤트(에러, 로그인, 배포 등) 자동 수집 및 보관 방법 모색 및 적용"
      ]
   }

In [ ]:
security_rule_monitoring_story_list = [{'title': '개인정보 마스킹 처리 방안 정의',
   'description': '개인정보 및 보안 정보를 마스킹 처리하기 위한 기준과 방법을 정의한다.',
   'acceptance_criteria': ['개인정보 마스킹 처리 기준이 문서화됨.',
    '마스킹 처리 방법이 모든 팀원에게 공유됨.',
    '정의된 기준에 따라 샘플 데이터가 마스킹 처리됨.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['개인정보 보호', '보안', '마스킹 처리']},
  {'title': '보안 정보 마스킹 처리 방안 정의',
   'description': '보안 정보를 마스킹 처리하기 위한 기준과 방법을 정의한다.',
   'acceptance_criteria': ['보안 정보 마스킹 처리 기준이 문서화됨.',
    '마스킹 처리 방법이 모든 팀원에게 공유됨.',
    '정의된 기준에 따라 샘플 보안 정보가 마스킹 처리됨.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['보안', '마스킹 처리', '정보 보호']},
  {'title': '마스킹 처리 기준 적용',
   'description': '정의된 개인정보 및 보안 정보 마스킹 처리 기준을 시스템에 적용한다.',
   'acceptance_criteria': ['모든 로그에서 개인정보가 마스킹 처리됨.',
    '마스킹 처리 기준이 시스템에 적용되어 정상 작동함.',
    '적용 후 테스트 결과가 기준에 부합함.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['개인정보 보호', '보안', '시스템 적용']},
  {'title': '마스킹 처리 기준 문서화',
   'description': '마스킹 처리 기준을 문서화하여 팀원들이 쉽게 접근할 수 있도록 한다.',
   'acceptance_criteria': ['마스킹 처리 기준이 문서화된 문서가 작성됨.',
    '문서가 팀원들에게 공유됨.',
    '문서에 대한 피드백이 수집되고 반영됨.'],
   'domain': 'documentation',
   'story_type': 'documentation',
   'tags': ['문서화', '개인정보 보호', '보안']}]

In [ ]:
security_rule_monitoring_epic_info = {
      "title":"보안 및 개인정보 보호 정책",
      "description":"개인정보 및 보안 정보를 마스킹 처리하는 방안을 통일하고 적용하는 작업.",
      "business_value":"사용자 개인정보 보호 및 보안 강화를 통해 법적 요구사항을 준수.",
      "priority":"High",
      "acceptance_criteria":[
         "개인정보 및 보안 정보의 마스킹 처리 방안이 통일됨.",
         "마스킹 처리 기준이 문서화되고 적용됨.",
         "모든 로그에서 개인정보가 안전하게 처리됨."
      ],
      "included_tasks":[
         "개인정보/보안 정보 마스킹 처리 방안 통일"
      ]
   }

In [73]:
docker_environment_story_list = [{'title': 'Dockerfile 멀티스테이지 빌드 구성',
   'description': '효율적인 이미지를 생성하기 위해 Dockerfile을 멀티스테이지 빌드 방식으로 구성한다.',
   'acceptance_criteria': ['Dockerfile이 멀티스테이지 빌드 방식으로 작성되어야 한다.',
    '최종 이미지 크기가 100MB 이하로 유지되어야 한다.',
    '빌드 과정에서 오류가 발생하지 않아야 한다.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['Docker', 'CI/CD', '빌드']},
  {'title': 'Docker Compose로 환경별 서비스 분리',
   'description': 'Docker Compose를 사용하여 개발, 테스트, 프로덕션 환경별로 서비스를 분리한다.',
   'acceptance_criteria': ['각 환경에 맞는 docker-compose.yml 파일이 생성되어야 한다.',
    '서비스가 환경에 따라 올바르게 시작되고 종료되어야 한다.',
    '환경별로 필요한 볼륨과 네트워크가 설정되어야 한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Docker', 'Docker Compose', '환경 관리']},
  {'title': '환경변수 관리 방식 설정',
   'description': '환경변수를 관리하기 위한 방식을 설정하고 문서화한다.',
   'acceptance_criteria': ['환경변수가 .env 파일 또는 Secrets Manager를 통해 관리되어야 한다.',
    'ConfigMap을 사용하여 Kubernetes 환경에서도 환경변수가 설정되어야 한다.',
    '환경변수 사용법에 대한 문서가 작성되어야 한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Docker', '환경변수', 'Kubernetes']}]

In [74]:
docker_environment_epic_info = {'title': 'Docker 환경 설정 및 관리',
  'description': 'Docker 환경을 설정하고 관리하기 위한 작업들을 포함합니다.',
  'business_value': '효율적인 개발 환경 구축 및 운영',
  'priority': 'High',
  'acceptance_criteria': ['Dockerfile 멀티스테이지 빌드가 성공적으로 구성되어야 한다.',
   'Docker Compose를 통해 환경별 서비스가 분리되어야 한다.',
   '환경변수 관리 방식이 설정되어야 한다.'],
  'included_tasks': ['Dockerfile 멀티스테이지 빌드 구성',
   'Docker Compose로 환경별 서비스 분리',
   '환경변수 관리 방식 설정( `.env`, Secrets Manager, ConfigMap 등)']}

In [79]:
container_network_story_list = [{'title': '컨테이너 네트워크 브릿지 구성',
   'description': '컨테이너 간의 통신을 위한 브릿지 네트워크를 설정한다.',
   'acceptance_criteria': ['브릿지 네트워크가 성공적으로 생성되어야 한다.',
    '컨테이너가 브릿지 네트워크를 통해 서로 통신할 수 있어야 한다.',
    '네트워크 설정이 문서화되어야 한다.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['컨테이너', '네트워크', 'Docker']},
  {'title': '포트 바인딩 설정',
   'description': '컨테이너의 외부 접근을 위한 포트 바인딩을 설정한다.',
   'acceptance_criteria': ['각 컨테이너에 대해 지정된 포트가 바인딩되어야 한다.',
    '외부에서 컨테이너에 접근할 수 있어야 한다.',
    '포트 바인딩 설정이 문서화되어야 한다.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['컨테이너', '네트워크', 'Docker']},
  {'title': 'DNS 구성',
   'description': '컨테이너 간의 이름 해석을 위한 DNS 구성을 완료한다.',
   'acceptance_criteria': ['DNS 서비스가 성공적으로 설정되어야 한다.',
    '컨테이너가 이름으로 서로를 인식할 수 있어야 한다.',
    'DNS 설정이 문서화되어야 한다.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['컨테이너', '네트워크', 'Docker']},
  {'title': '볼륨 마운트 설정',
   'description': '데이터 영속성을 위한 볼륨 마운트를 설정한다.',
   'acceptance_criteria': ['각 컨테이너에 대해 볼륨이 성공적으로 마운트되어야 한다.',
    '데이터가 컨테이너 재시작 후에도 유지되어야 한다.',
    '볼륨 마운트 설정이 문서화되어야 한다.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['데이터', '영속성', 'Docker']},
  {'title': '데이터 영속화 정책 수립',
   'description': '데이터 영속성을 위한 정책을 수립하고 문서화한다.',
   'acceptance_criteria': ['데이터 영속화 정책이 정의되어야 한다.',
    '정책이 팀원들과 공유되어야 한다.',
    '정책이 문서화되어야 한다.'],
   'domain': 'data',
   'story_type': 'documentation',
   'tags': ['데이터', '영속성', '정책']}]

In [80]:
container_network_epic_info = {'title': '컨테이너 네트워크 및 데이터 관리',
  'description': '컨테이너의 네트워크 및 데이터 영속성을 관리하는 작업들을 포함합니다.',
  'business_value': '안정적인 데이터 관리 및 네트워크 구성',
  'priority': 'Medium',
  'acceptance_criteria': ['컨테이너 네트워크가 브릿지, 포트 바인딩, DNS 구성이 완료되어야 한다.',
   '볼륨 마운트 및 데이터 영속화 정책이 수립되어야 한다.'],
  'included_tasks': ['컨테이너 네트워크 브릿지, 포트 바인딩, DNS 구성', '볼륨 마운트 및 데이터 영속화 정책']}

In [85]:
kubernetes_cluster_story_list = [{'title': 'Kubernetes 적용 여부 판단',
   'description': 'Kubernetes 적용 여부를 판단하고, 적용할 경우 클러스터 구성을 위한 준비 작업을 진행한다.',
   'acceptance_criteria': ['Kubernetes 적용 여부에 대한 결정을 문서화한다.',
    '적용 시 필요한 리소스 목록을 작성한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Kubernetes', '클러스터 구성', 'devops']},
  {'title': 'Kubernetes 클러스터 구성',
   'description': 'eksctl을 사용하여 Kubernetes 클러스터를 구성한다.',
   'acceptance_criteria': ['eksctl을 사용하여 클러스터가 성공적으로 생성되어야 한다.',
    "클러스터의 상태가 'ACTIVE'로 표시되어야 한다."],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Kubernetes', 'eksctl', '클러스터 구성']},
  {'title': 'Deployment 리소스 정의',
   'description': 'Kubernetes 클러스터에 배포할 애플리케이션의 Deployment 리소스를 정의한다.',
   'acceptance_criteria': ['Deployment YAML 파일이 작성되어야 한다.',
    'Deployment가 성공적으로 생성되어야 한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Kubernetes', 'Deployment', '리소스 정의']},
  {'title': 'Service 리소스 정의',
   'description': 'Kubernetes 클러스터에서 애플리케이션에 접근할 수 있도록 Service 리소스를 정의한다.',
   'acceptance_criteria': ['Service YAML 파일이 작성되어야 한다.',
    'Service가 성공적으로 생성되어야 한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Kubernetes', 'Service', '리소스 정의']},
  {'title': 'Ingress 리소스 정의',
   'description': 'Kubernetes 클러스터에서 외부 트래픽을 관리하기 위한 Ingress 리소스를 정의한다.',
   'acceptance_criteria': ['Ingress YAML 파일이 작성되어야 한다.',
    'Ingress가 성공적으로 생성되어야 한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Kubernetes', 'Ingress', '리소스 정의']},
  {'title': 'Helm Chart 템플릿화',
   'description': '애플리케이션 배포를 위한 Helm Chart를 템플릿화한다.',
   'acceptance_criteria': ['Helm Chart가 성공적으로 생성되어야 한다.',
    'Helm Chart를 사용하여 Deployment, Service, Ingress가 배포 가능해야 한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Kubernetes', 'Helm', '템플릿화']},
  {'title': 'HPA 적용',
   'description': 'Horizontal Pod Autoscaler(HPA)를 클러스터에 적용하여 자동 스케일링을 설정한다.',
   'acceptance_criteria': ['HPA 리소스가 성공적으로 생성되어야 한다.',
    'HPA가 정상적으로 작동하여 Pod 수가 자동으로 조정되어야 한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Kubernetes', 'HPA', '자동 스케일링']}]

In [87]:
kubernetes_cluster_epic_info =    {
      "title":"Kubernetes 클러스터 구성 및 관리 또는 Kubernetes 미적용",
      "description":"Kubernetes 클러스터를 구성하고 관리하기 위한 작업들을 포함합니다.",
      "business_value":"확장성과 관리 용이성을 제공하는 클라우드 네이티브 환경 구축",
      "priority":"High",
      "acceptance_criteria":[
         "Kubernetes 적용 여부가 판단되어야 하며, 적용 시 클러스터가 성공적으로 구성되어야 한다.",
         "Deployment, Service, Ingress 등의 리소스 정의가 완료되어야 한다.",
         "Helm Chart로 템플릿화가 완료되어야 하며, HPA가 적용되어야 한다."
      ],
      "included_tasks":[
         "Kubernetes 적용 여부 판단 후 적용 또는 미적용(`eksctl`, `Helm` 사용한 클러스터 구성, Deployment, Service, Ingress 등 리소스 정의, Helm Chart로 템플릿화, HPA (Horizontal Pod Autoscaler) 적용 등)"
      ]
   }

In [93]:
deploy_policy_story_list = [{'title': '배포 리소스 네이밍 가이드 작성',
   'description': 'Helm Chart 및 Kubernetes 리소스에 대한 네이밍 가이드를 작성하여 일관된 배포 리소스 관리를 지원한다.',
   'acceptance_criteria': ['Helm Chart에 대한 네이밍 규칙이 문서화되어야 한다.',
    'Kubernetes 리소스에 대한 네이밍 규칙이 문서화되어야 한다.',
    '작성된 가이드가 팀원들에게 공유되어야 한다.'],
   'domain': 'devops',
   'story_type': 'documentation',
   'tags': ['배포 정책', 'Kubernetes', 'Helm']},
  {'title': 'Secret 및 ConfigMap 분리 저장 기준 수립',
   'description': 'Secret과 ConfigMap을 분리 저장하고 암호화하는 기준을 수립하여 보안성을 강화한다.',
   'acceptance_criteria': ['Secret과 ConfigMap의 저장 방식에 대한 기준이 문서화되어야 한다.',
    '암호화 방법 및 절차가 명시되어야 한다.',
    '기준이 팀원들에게 교육되어야 한다.'],
   'domain': 'devops',
   'story_type': 'documentation',
   'tags': ['보안 관리', 'Secret', 'ConfigMap']},
  {'title': '이미지 취약점 검사 및 업데이트 주기 문서화',
   'description': '배포 이미지에 대한 취약점 검사 및 업데이트 주기를 문서화하여 안전한 배포 환경을 구축한다.',
   'acceptance_criteria': ['이미지 취약점 검사 주기가 명시되어야 한다.',
    '업데이트 주기가 문서화되어야 한다.',
    '문서가 팀원들에게 공유되어야 한다.'],
   'domain': 'devops',
   'story_type': 'documentation',
   'tags': ['배포 정책', '이미지 관리', '보안']}]

In [94]:
deploy_policy_epic_info =  {'title': '자동 배포 시스템 구축',
  'description': '이미지 자동 배포 방식을 모색하고 설정하는 작업을 포함합니다.',
  'business_value': '배포 효율성 향상 및 인적 오류 감소',
  'priority': 'High',
  'acceptance_criteria': ['이미지 자동 배포 방식이 성공적으로 설정되어야 한다.'],
  'included_tasks': ['이미지 자동 배포방식 모색 및 설정']}

In [102]:
auto_infra_story_list = [{'title': '기존 AWS 리소스 Terraform으로 코드화',
   'description': '기존 AWS 리소스를 Terraform 코드로 변환하여 인프라를 선언형으로 관리할 수 있도록 한다.',
   'acceptance_criteria': ['모든 기존 AWS 리소스가 Terraform 코드로 변환 완료',
    '변환된 Terraform 코드가 AWS 환경에서 정상 작동하는지 검증 완료',
    '코드화된 리소스에 대한 문서화 완료'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Terraform', 'AWS', '인프라 관리']},
  {'title': 'CI/CD 연동을 통한 IaC 자동 적용 프로세스 구축',
   'description': 'CI/CD 파이프라인을 설정하여 Terraform 코드 변경 시 자동으로 인프라 변경이 적용되도록 한다.',
   'acceptance_criteria': ['CI/CD 파이프라인이 성공적으로 구축됨',
    'Terraform 코드 변경 시 자동으로 인프라 변경이 적용되는지 확인',
    '변경 사항에 대한 알림 시스템이 작동하는지 검증'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['CI/CD', 'Terraform', '자동화']},
  {'title': 'PR 생성 시 terraform plan 결과 리뷰 기능 구현',
   'description': 'PR 생성 시 Terraform plan 결과를 자동으로 리뷰할 수 있는 기능을 구현한다.',
   'acceptance_criteria': ['PR 생성 시 terraform plan 결과가 자동으로 생성됨',
    '리뷰어가 terraform plan 결과를 확인할 수 있는 UI 제공',
    '리뷰어가 결과에 대해 피드백을 남길 수 있는 기능 구현'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Terraform', 'PR 리뷰', '자동화']},
  {'title': '테스트 환경 자동 배포 기능 구현',
   'description': 'PR 생성 후 테스트 환경(Sandbox)을 자동으로 배포하는 기능을 구현한다.',
   'acceptance_criteria': ['PR 생성 시 자동으로 테스트 환경이 배포됨',
    '테스트 환경에서의 정상 작동 여부를 확인할 수 있는 로그 제공',
    '테스트 환경 배포 후 smoke test가 자동으로 실행됨'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Terraform', '테스트 자동화', 'CI/CD']},
  {'title': 'smoke test 자동화 기능 구현',
   'description': '테스트 환경 배포 후 자동으로 smoke test를 실행하는 기능을 구현한다.',
   'acceptance_criteria': ['smoke test가 자동으로 실행됨',
    'smoke test 결과가 CI/CD 대시보드에 표시됨',
    'smoke test 실패 시 알림이 전송됨'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['테스트 자동화', 'CI/CD', 'Terraform']},
  {'title': '인프라 변경 승인 후 apply 진행 기능 구현',
   'description': 'smoke test가 성공적으로 완료된 후 인프라 변경을 승인하고 apply를 진행하는 기능을 구현한다.',
   'acceptance_criteria': ['smoke test 성공 후 apply 진행 버튼이 활성화됨',
    'apply 진행 시 변경 사항에 대한 로그가 기록됨',
    'apply 완료 후 결과가 CI/CD 대시보드에 표시됨'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Terraform', 'CI/CD', '자동화']}]

In [103]:
auto_infra_epic_info = {
      "title":"인프라 관리 자동화",
      "description":"AWS 리소스를 Terraform 또는 AWS CDK로 관리하고, CI/CD 연동을 통해 인프라 변경을 자동화하는 Epic입니다.",
      "business_value":"인프라 관리의 효율성을 높이고, 변경 사항의 안정성을 확보합니다.",
      "priority":"High",
      "acceptance_criteria":[
         "기존 AWS 리소스를 Terraform 또는 AWS CDK로 코드화 완료",
         "CI/CD 연동을 통해 IaC 자동 적용 프로세스 구축",
         "PR 생성 시 terraform plan 결과 리뷰 및 테스트 환경 자동 배포 기능 구현"
      ],
      "included_tasks":[
         "기존 AWS 리소스를 포함하여 인프라를 선언형 코드로 관리하도록 전환",
         "CI/CD 연동으로 IaC 자동 적용",
         "PR 생성 시 terraform plan 결과 리뷰 → 테스트 환경 자동 배포(Sandbox) → smoke test → 승인 후 apply 진행"
      ]
   }

In [106]:
module_management_story_list = [{'title': '환경별 Workspace 분리 적용',
   'description': '개발, 스테이징, 프로덕션 환경에 대한 Workspace를 분리하여 관리의 일관성을 높인다.',
   'acceptance_criteria': ['dev, staging, prod 환경에 대한 Workspace 분리 적용 완료',
    '각 환경별 설정 파일이 정상적으로 작동하는지 확인'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Workspace', '환경 관리', '인프라 코드']},
  {'title': '모듈화 적용',
   'description': '인프라 코드를 모듈화하여 코드의 재사용성을 높인다.',
   'acceptance_criteria': ['모듈화된 코드가 각 환경에서 정상적으로 작동하는지 확인',
    '모듈화된 코드에 대한 문서화 완료'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['모듈화', '코드 재사용', '인프라 코드']},
  {'title': '디렉토리 구조 표준화',
   'description': '인프라 코드의 디렉토리 구조를 표준화하여 관리의 일관성을 높인다.',
   'acceptance_criteria': ['디렉토리 구조가 표준화된 가이드에 맞춰 작성됨',
    '기존 코드가 새로운 디렉토리 구조에 맞게 리팩토링 완료'],
   'domain': 'devops',
   'story_type': 'enhancement',
   'tags': ['디렉토리 구조', '표준화', '인프라 코드']},
  {'title': '변수 표준화',
   'description': '인프라 코드에서 사용하는 변수를 표준화하여 관리의 일관성을 높인다.',
   'acceptance_criteria': ['모든 변수명이 표준화된 가이드에 맞춰 변경됨', '변수 사용에 대한 문서화 완료'],
   'domain': 'devops',
   'story_type': 'enhancement',
   'tags': ['변수', '표준화', '인프라 코드']},
  {'title': '출력 표준화',
   'description': '인프라 코드의 출력 형식을 표준화하여 일관성을 높인다.',
   'acceptance_criteria': ['모든 출력 형식이 표준화된 가이드에 맞춰 작성됨', '출력 형식에 대한 문서화 완료'],
   'domain': 'devops',
   'story_type': 'enhancement',
   'tags': ['출력', '표준화', '인프라 코드']},
  {'title': '가이드 문서화',
   'description': '모듈화 및 환경별 Workspace 분리에 대한 가이드를 문서화한다.',
   'acceptance_criteria': ['모듈화 및 Workspace 분리에 대한 가이드 문서 작성 완료',
    '가이드 문서가 팀원들에게 공유됨'],
   'domain': 'documentation',
   'story_type': 'documentation',
   'tags': ['문서화', '가이드', '인프라 코드']}]

In [107]:
module_management_epic_info =    {
      "title":"모듈화 및 환경 관리",
      "description":"인프라 코드의 모듈화 및 환경별 Workspace 분리를 통해 관리의 일관성을 높이는 Epic입니다.",
      "business_value":"환경별 관리의 용이성을 제공하고, 코드의 재사용성을 높입니다.",
      "priority":"Medium",
      "acceptance_criteria":[
         "모듈화 및 환경별 Workspace 분리 적용 완료",
         "디렉토리 구조/변수/출력 표준화 가이드 문서화 완료"
      ],
      "included_tasks":[
         "모듈화 및 환경별 Workspace 분리 적용 (dev, staging, prod) 및 디렉토리 구조/변수/출력 표준화 가이드 문서화"
      ]
   }

In [110]:
security_policy_iac_by_story_list = [{'title': 'IAM Role 및 Policy 정의 자동화',
   'description': 'IAM Role 및 Policy를 코드로 정의하고 관리하는 자동화 스크립트를 작성한다.',
   'acceptance_criteria': ['IAM Role 및 Policy가 코드로 정의되어 Git 저장소에 커밋됨',
    '정의된 IAM Role 및 Policy가 AWS 환경에 성공적으로 배포됨',
    '정의된 IAM Role 및 Policy에 대한 문서화가 완료됨'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['IAM', 'AWS', '보안']},
  {'title': '민감 정보 관리 자동화',
   'description': '민감 정보를 AWS Secrets Manager 및 SSM Parameter Store로 분리하고 Secret 자동 Rotate 정책을 적용한다.',
   'acceptance_criteria': ['민감 정보가 AWS Secrets Manager 및 SSM Parameter Store에 성공적으로 저장됨',
    'Secret 자동 Rotate 정책이 설정되어 정상 작동함',
    '민감 정보 접근 로그가 CloudTrail에 기록됨'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['AWS Secrets Manager', 'SSM', '보안']},
  {'title': 'Security Group 및 NACL 설정 자동화',
   'description': 'Security Group 및 NACL 설정을 코드로 정의하고 자동으로 배포하는 스크립트를 작성한다.',
   'acceptance_criteria': ['Security Group 및 NACL 설정이 코드로 정의되어 Git 저장소에 커밋됨',
    '정의된 Security Group 및 NACL이 AWS 환경에 성공적으로 배포됨',
    '보안 설정에 대한 문서화가 완료됨'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['Security Group', 'NACL', 'AWS', '보안']},
  {'title': 'S3 버킷 정책 자동화',
   'description': 'S3 버킷의 보안 정책을 코드로 정의하고 자동으로 적용하는 스크립트를 작성한다.',
   'acceptance_criteria': ['S3 버킷 정책이 코드로 정의되어 Git 저장소에 커밋됨',
    '정의된 S3 버킷 정책이 AWS 환경에 성공적으로 배포됨',
    'S3 버킷 정책에 대한 문서화가 완료됨'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['S3', 'AWS', '보안']}]

In [111]:
security_policy_by_iac_epic_info =    {
      "title":"보안 정책 및 관리",
      "description":"인프라의 보안을 강화하기 위한 정책을 정의하고 자동화하는 Epic입니다.",
      "business_value":"보안 리스크를 줄이고, 인프라의 안전성을 높입니다.",
      "priority":"High",
      "acceptance_criteria":[
         "IAM Role 및 Policy 정의를 코드로 관리 완료",
         "민감 정보는 AWS Secrets Manager / SSM Parameter Store로 분리 및 Secret 자동 Rotate 정책 적용"
      ],
      "included_tasks":[
         "보안 정책 내재화 (IAM Role 및 Policy 정의를 코드로 관리, Security Group, NACL, S3 정책 등 보안 설정 자동화)"
      ]
   }

In [118]:
policy_rule_iac_story_list = [{'title': 'IaC 리소스 명명 규칙 정의',
   'description': '인프라 리소스의 명명 규칙을 정의하고 문서화한다.',
   'acceptance_criteria': ['명명 규칙 문서가 작성되고 팀에 공유됨',
    '명명 규칙이 모든 관련 인프라 리소스에 적용됨',
    '명명 규칙에 대한 팀 피드백 수집 및 반영 완료'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['IaC', '명명 규칙', '인프라 관리']},
  {'title': '접근 권한 분리 정책 정의',
   'description': '인프라 리소스에 대한 접근 권한 분리 정책을 수립한다.',
   'acceptance_criteria': ['접근 권한 분리 정책 문서가 작성되고 팀에 공유됨',
    '정의된 정책이 모든 관련 인프라 리소스에 적용됨',
    '정기적인 접근 권한 검토 주기 정의 완료'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['접근 권한', '정책 관리', '인프라 관리']},
  {'title': '정기적인 모듈 업데이트 주기 정의',
   'description': '모듈 업데이트 및 리팩토링을 위한 정기적인 주기를 정의한다.',
   'acceptance_criteria': ['모듈 업데이트 주기가 문서화되고 팀에 공유됨',
    '정기적인 업데이트 일정이 팀 캘린더에 반영됨',
    '업데이트 주기에 대한 팀 피드백 수집 및 반영 완료'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['모듈 업데이트', '리팩토링', '인프라 관리']}]

In [119]:
policy_rule_iac_epic_info =    {
      "title":"정책 및 규칙 관리",
      "description":"인프라 리소스의 명명 규칙 및 접근 권한을 정의하고 관리하는 Epic입니다.",
      "business_value":"조직 내 인프라 관리의 일관성을 높입니다.",
      "priority":"Low",
      "acceptance_criteria":[
         "IaC 리소스 명명 규칙 및 접근 권한 분리 정책 정의 완료",
         "정기적인 모듈 업데이트 및 리팩토링 주기 정의 완료"
      ],
      "included_tasks":[
         "정책 생성 (IaC 리소스 명명 규칙, 접근 권한 분리 등)"
      ]
   }

In [122]:
branch_protect_story_list = [{'title': '브랜치 병합 제한 설정',
   'description': 'main 및 develop 브랜치에 대한 병합 제한 규칙을 설정한다.',
   'acceptance_criteria': ['main 브랜치에 대한 PR 병합이 제한된다.',
    'develop 브랜치에 대한 PR 병합이 제한된다.',
    '병합 제한 설정이 GitHub 또는 GitLab에서 확인 가능하다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['브랜치 보호', 'Git', 'CI/CD']},
  {'title': '브랜치 보호 규칙 적용',
   'description': '브랜치 보호 규칙을 설정하여 코드 리뷰 및 상태 확인을 적용한다.',
   'acceptance_criteria': ['리뷰 승인 규칙이 main 및 develop 브랜치에 적용된다.',
    '상태 확인 규칙이 main 및 develop 브랜치에 적용된다.',
    '브랜치 보호 규칙이 GitHub 또는 GitLab에서 확인 가능하다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['브랜치 보호', '코드 리뷰', 'CI/CD']},
  {'title': '환경 및 비밀 설정 구성',
   'description': '각 환경별로 .env 파일, Secrets Manager, Parameter Store를 통해 비밀 정보를 안전하게 관리한다.',
   'acceptance_criteria': ['.env 파일이 각 환경에 맞게 구성된다.',
    'Secrets Manager에 비밀 정보가 안전하게 저장된다.',
    'Parameter Store에 환경 변수들이 안전하게 관리된다.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['환경 관리', '비밀 관리', 'AWS']},
  {'title': '환경별 배포 구성 문서화',
   'description': '각 환경별 배포 구성 및 변수 관리에 대한 문서를 작성한다.',
   'acceptance_criteria': ['각 환경별 배포 구성 문서가 작성된다.',
    '변수 관리에 대한 문서가 작성된다.',
    '문서가 팀원들에게 공유되고 피드백을 받는다.'],
   'domain': 'documentation',
   'story_type': 'documentation',
   'tags': ['문서화', '배포', '환경 관리']}]

In [123]:
branch_protect_epic_info = {
	  "title": "브랜치 보호 및 환경/비밀 관리",
	  "description": "코드 품질을 보장하기 위한 브랜치 보호 설정과, 각 환경별 구성 및 민감 정보의 안전한 관리를 수행한다.",
	  "business_value": "협업 효율성과 코드 안정성을 높이고, 환경별 배포 신뢰성을 강화한다.",
	  "priority": "Medium",
	  "acceptance_criteria": [
	    "PR 병합은 main, develop 브랜치로 제한된다.",
	    "브랜치 보호 규칙이 적용되어 있다.",
	    "각 브랜치/환경에 대해 별도 설정 파일 및 비밀 저장소가 관리된다."
	  ],
	  "included_tasks": [
	    "main, develop 브랜치에 대한 병합 제한 규칙 설정",
	    "브랜치 보호 규칙(리뷰 승인, 상태 확인 등) 적용",
	    ".env 파일, Secrets Manager, Parameter Store를 통한 환경 및 비밀 설정 구성",
	    "환경별 배포 구성 및 변수 관리 문서화"
	  ]
 }

In [124]:
docker_images_story_list = [{'title': 'CodeBuild 연동 설정',
   'description': 'Docker 이미지를 빌드하기 위해 CodeBuild와 연동하는 설정을 구성한다.',
   'acceptance_criteria': ['CodeBuild 프로젝트가 생성되고, buildspec.yml 파일이 올바르게 설정된다.',
    'CodeBuild가 성공적으로 실행되어 Docker 이미지가 빌드된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Docker', 'CI/CD', 'CodeBuild']},
  {'title': 'DockerHub에 이미지 푸시 기능 구현',
   'description': '빌드된 Docker 이미지를 DockerHub에 자동으로 푸시하는 기능을 구현한다.',
   'acceptance_criteria': ['DockerHub에 로그인할 수 있는 자격증명이 설정된다.',
    '빌드가 완료된 후 Docker 이미지가 DockerHub에 자동으로 푸시된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Docker', 'CI/CD', 'DockerHub']},
  {'title': 'ECR에 이미지 푸시 기능 구현',
   'description': '빌드된 Docker 이미지를 AWS ECR에 자동으로 푸시하는 기능을 구현한다.',
   'acceptance_criteria': ['ECR 리포지토리가 생성되고, 필요한 권한이 설정된다.',
    '빌드가 완료된 후 Docker 이미지가 ECR에 자동으로 푸시된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['Docker', 'CI/CD', 'ECR']}]

In [125]:
docker_images_epic_info =   {
    "title": "Docker 이미지 관리",
    "description": "CI/CD 프로세스에서 Docker 이미지를 자동으로 빌드하고 푸시하는 기능을 구현한다.",
    "business_value": "애플리케이션 배포의 일관성을 유지하고, 배포 속도를 향상시킨다.",
    "priority": "High",
    "acceptance_criteria": [
      "CodeBuild와 연동하여 buildspec.yml이 사용된다.",
      "DockerHub 또는 ECR에 결과 이미지가 자동으로 푸시된다."
    ],
    "included_tasks": [
      "CodeBuild 연동 (buildspec.yml)",
      "DockerHub 또는 ECR에 결과 이미지 자동 푸시"
    ]
  }

In [126]:
deploy_stable_story_list = [{'title': '배포 트리거 전략 수립 및 적용',
   'description': 'GitHub Webhook, PR 병합, 수동 승인 등을 통해 배포 트리거를 설정한다.',
   'acceptance_criteria': ['GitHub Webhook이 성공적으로 설정되어 배포가 자동으로 트리거된다.',
    'PR 병합 시 배포가 자동으로 시작된다.',
    '수동 승인 프로세스가 구현되어 배포가 승인 후에만 진행된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['배포 전략', 'GitHub', '자동화']},
  {'title': '배포 전략 적용',
   'description': '블루그린, 카나리아, 롤링 중 하나 이상의 배포 전략을 적용한다.',
   'acceptance_criteria': ['블루그린 배포 전략이 성공적으로 구현된다.',
    '카나리아 배포 전략이 성공적으로 구현된다.',
    '롤링 배포 전략이 성공적으로 구현된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['배포 전략', '신뢰성', '운영 안정성']},
  {'title': '안전한 배포 구성',
   'description': 'AWS IAM Role 및 SSH 키를 통해 안전한 배포 환경을 구성한다.',
   'acceptance_criteria': ['AWS IAM Role이 성공적으로 생성되고 필요한 권한이 부여된다.',
    'SSH 키가 안전하게 생성되고 배포 서버에 적용된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['보안', 'AWS', '배포']},
  {'title': '헬스체크 구현',
   'description': '배포 전후에 헬스체크를 수행하고, 실패 시 자동 롤백이 작동하도록 설정한다.',
   'acceptance_criteria': ['배포 전 헬스체크가 성공적으로 수행된다.',
    '배포 후 헬스체크가 성공적으로 수행된다.',
    '헬스체크 실패 시 자동 롤백이 작동한다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['헬스체크', '자동 롤백', '배포 안정성']},
  {'title': '배포 전략 및 롤백 구성 문서화',
   'description': '배포 전략 및 롤백 구성에 대한 문서를 작성한다.',
   'acceptance_criteria': ['배포 전략에 대한 문서가 작성되어 팀과 공유된다.',
    '롤백 프로세스에 대한 문서가 작성되어 팀과 공유된다.'],
   'domain': 'documentation',
   'story_type': 'documentation',
   'tags': ['문서화', '배포 전략', '팀 공유']}]

In [127]:
deploy_stable_epic_info =   {
  "title": "배포 전략 및 운영 안정성 관리",
  "description": "다양한 배포 전략, 보안 설정, 헬스체크 및 롤백을 통합하여 운영 중단 없이 안전하고 신뢰성 있는 배포를 지원한다.",
  "business_value": "서비스 중단 없는 배포를 가능하게 하여 사용자 경험을 보호하고, 운영 리스크를 최소화한다.",
  "priority": "High",
  "acceptance_criteria": [
    "배포 트리거가 GitHub Webhook, PR 병합, 수동 승인 등으로 설정된다.",
    "배포 전략이 블루그린, 카나리아, 롤링 중 하나 이상으로 설정된다.",
    "AWS IAM Role 및 SSH 키를 통한 보안 배포가 구성된다.",
    "배포 전후 헬스체크가 수행되고, 실패 시 자동 롤백이 작동한다."
  ],
  "included_tasks": [
    "배포 트리거 전략 수립 및 적용(GitHub Webhook, 수동 승인 등)",
    "배포 전략 적용(블루그린, 카나리아, 롤링 중 선택)",
    "AWS IAM Role 및 SSH 키를 통한 안전한 배포 구성",
    "배포 전/후 헬스체크 구현 및 실패 시 자동 롤백 설정",
    "배포 전략 및 롤백 구성 관련 문서화"
  ]
}

In [128]:
deploy_policy_story_list = [{'title': '배포 이력 및 승인 기록 저장 시스템 구성',
   'description': '배포 이력과 승인 기록을 저장하고 관리할 수 있는 시스템을 구축한다.',
   'acceptance_criteria': ['배포 이력과 승인 기록이 데이터베이스에 저장된다.',
    '저장된 기록은 사용자 인터페이스를 통해 조회 가능하다.',
    '기록 저장 시 오류가 발생하지 않는다.'],
   'domain': 'backend',
   'story_type': 'feature',
   'tags': ['배포 이력', '데이터베이스', '운영 관리']},
  {'title': '운영 및 배포 프로세스에 대한 정책 문서화',
   'description': '운영 및 배포 프로세스를 문서화하여 팀원들이 쉽게 접근할 수 있도록 한다.',
   'acceptance_criteria': ['운영 및 배포 프로세스에 대한 문서가 작성된다.',
    '문서는 팀의 위키 또는 문서 저장소에 업로드된다.',
    '문서의 내용이 팀원들에 의해 검토되고 승인된다.'],
   'domain': 'documentation',
   'story_type': 'documentation',
   'tags': ['운영 정책', '문서화', '프로세스 관리']},
  {'title': '워크플로 및 파이프라인 명명 규칙 정의',
   'description': '배포 워크플로 및 파이프라인의 명명 규칙을 정의하여 일관성을 유지한다.',
   'acceptance_criteria': ['명명 규칙이 문서화되어 팀원들에게 공유된다.',
    '명명 규칙에 따라 기존 워크플로와 파이프라인이 업데이트된다.',
    '명명 규칙이 팀원들에 의해 준수된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['명명 규칙', '워크플로', '파이프라인']},
  {'title': '신규 프로젝트용 배포 템플릿 및 체크리스트 작성',
   'description': '신규 프로젝트에 사용할 수 있는 배포 정책 템플릿과 체크리스트를 작성한다.',
   'acceptance_criteria': ['배포 템플릿이 작성되고 문서화된다.',
    '체크리스트가 작성되어 팀원들에게 배포된다.',
    '템플릿과 체크리스트가 실제 프로젝트에 적용 가능하다.'],
   'domain': 'frontend',
   'story_type': 'feature',
   'tags': ['배포 템플릿', '체크리스트', '프로젝트 관리']}]

In [ ]:
deploy_policy_log  =   {
  "title": "운영 정책 및 배포 이력 관리",
  "description": "배포 정책과 템플릿을 표준화하고, 배포 이력 및 승인 기록을 체계적으로 관리하여 프로젝트의 일관성과 추적 가능성을 확보한다.",
  "business_value": "운영 프로세스를 표준화하여 품질을 유지하고, 문제 발생 시 신속한 원인 파악 및 대응이 가능하도록 한다.",
  "priority": "Medium",
  "acceptance_criteria": [
    "배포 이력 및 승인 내역이 저장되고 관리된다.",
    "워크플로 및 파이프라인의 명명 규칙이 명확히 정의되어 있다.",
    "신규 프로젝트에 사용할 수 있는 배포 정책 템플릿과 체크리스트가 제공된다."
  ],
  "included_tasks": [
    "배포 이력 및 승인 기록 저장 시스템 구성",
    "운영 및 배포 프로세스에 대한 정책 문서화",
    "워크플로 및 파이프라인 명명 규칙 정의",
    "신규 프로젝트용 배포 템플릿 및 체크리스트 작성 및 배포"
  ]
}

In [136]:
story_point_by_story = {}

for story in deploy_policy_story_list[:]:
    story_point_by_story[story['title']] = {}

    estimated_story_point_str = story_point_agent._generate_estimations_with_llm(
        prompt = STORY_POINT_ESTIMATION_PROMPT,
        user_input = '',
        story_info= story,
        epic_info=deploy_policy_log,
        reference_stories=story_point_agent.get_reference_stories_by_domain('DevOps')
    )
    parsed = json.loads(estimated_story_point_str)
    story_point_by_story[story['title']] = parsed

INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료
INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료
INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료
INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료


In [138]:
story_point_by_story

{'배포 이력 및 승인 기록 저장 시스템 구성': [{'story_title': '배포 이력 및 승인 기록 저장 시스템 구성',
   'estimated_point': 5,
   'domain': 'backend',
   'estimation_method': 'cross_area',
   'reasoning': '배포 이력과 승인 기록을 저장하고 관리하는 시스템 구축은 데이터베이스 설계 및 사용자 인터페이스 개발이 포함되어 있어 복잡도가 중간 이상이다. 특히, 데이터베이스와의 연동 및 오류 처리 로직이 필요하므로 5 포인트로 추정하였다.',
   'complexity_factors': ['기술적 복잡도: 데이터베이스 설계 및 구현, 사용자 인터페이스 개발',
    '비즈니스 복잡도: 배포 이력 및 승인 기록의 명확한 요구사항',
    '위험 요소: 데이터 저장 시 오류 발생 가능성'],
   'similar_stories': ['코드 빌드 자동화 및 Docker 이미지 테스트',
    'DockerHub로 CI 결과물 자동 푸시'],
   'confidence_level': 'medium',
   'assumptions': ['데이터베이스와 사용자 인터페이스 간의 연동이 원활하게 이루어진다고 가정',
    '기존 시스템과의 통합에 큰 문제가 없을 것이라고 가정'],
   'risks': ['데이터베이스 설계 오류로 인한 데이터 손실 위험',
    '사용자 인터페이스에서의 오류 처리 미비로 인한 사용자 불만']}],
 '운영 및 배포 프로세스에 대한 정책 문서화': [{'story_title': '운영 및 배포 프로세스에 대한 정책 문서화',
   'estimated_point': 3,
   'domain': 'documentation',
   'estimation_method': 'cross_area',
   'reasoning': '운영 및 배포 프로세스를 문서화하는 작업은 팀원들이 쉽게 접근할 수 있도록 하는 것이며, 문서 작성과 검토 과정이 포함